In [11]:
import os
from langchain.llms import OpenAI
import langchain
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
import re
from youtube_transcript_api import YouTubeTranscriptApi
import openai
import gradio as gr
from elevenlabs import generate, play, set_api_key
import yaml

In [12]:
try:
    with open('secrets.yaml', 'r') as file:
        secrets = yaml.safe_load(file)

    OpenAI_Key_from_secrets_file = secrets['OpenAI_Key']
    ElevenLabs_key_from_secrets_file= secrets['ElevenLabs_key']

except:
    warning= "Please give your OpenAI and ElevenLabs credentials below as you dont have a secrets.yaml file"
    print(warning)

In [13]:
OpenAI_Key= OpenAI_Key_from_secrets_file           # Specify your key in quatation if you dont have a secrets.yaml file. for eg "sk-....7n"
ElevenLabs_key= ElevenLabs_key_from_secrets_file   # Specify your key in quatation if you dont have a secrets.yaml file. for eg "832...8n"

os.environ["OPENAI_API_KEY"] = OpenAI_Key
OpenAI.api_key = OpenAI_Key
openai.api_key = OpenAI_Key
set_api_key(ElevenLabs_key)

In [14]:
class DBSetup:

    def __init__(self):
        self.db= None
        self.chain= None
        self.vid_ID= None
        self.OpenAI_Key= None
        self.ElevenLabs_key= None

    # def API_setter(self):
    #     with open('secrets.yaml', 'r') as file:
    #         secrets = yaml.safe_load(file)

    #     self.OpenAI_Key = secrets['OpenAI_Key']
    #     self.ElevenLabs_key= secrets['ElevenLabs_key']
    #     os.environ["OPENAI_API_KEY"] = self.OpenAI_Key
    #     OpenAI.api_key = self.OpenAI_Key
    #     openai.api_key = self.OpenAI_Key
    #     set_api_key(self.ElevenLabs_key)

    def YT_transcript(self, url):

        def extract_video_id(url):
            pattern = re.compile(
                r'(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|\S*?[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})')
            match = pattern.search(url)
            return match.group(1) if match else None

        try:
            video_id = extract_video_id(url)
            if video_id:
                transcript = YouTubeTranscriptApi.get_transcript(video_id)
                text_only = ' '.join(item['text'] for item in transcript)
                self.vid_ID = video_id
                with open(f'Data_folder\{video_id}.txt', 'w') as file:
                    file.write(text_only)
            
            else:
                print("Error: Unable to extract transcript from the URL.")
                return None
        except Exception as e:
            print(f"Error getting transcript: {e}")
            return None


    def VectorDB(self):
        loader = TextLoader(f"Data_folder\{self.vid_ID}.txt")
        documents = loader.load()
        print(documents)

        # split it into chunks
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        docs = text_splitter.split_documents(documents)
        print("This from VectorDB fn:    ", docs)
        # create the open-source embedding function
        embedding_function = OpenAIEmbeddings()
        # load it into Chroma
        
        self.db =  Chroma.from_documents(docs, embedding_function, persist_directory=f'VectorDB/{self.vid_ID}')
        # _openai = OpenAI(model="gpt-3.5-turbo", temperature=0.7, max_tokens=3000)
        # self.chain = load_qa_chain(_openai, chain_type="stuff")


    def __chat_with_memory(self, full_prompt):

        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=[
            {"role": "system", "content": "You are a helpful assistant. Only answer if question is related to the context,if you don't know the  say I dont have any information regarding this topic"},
            {"role": "user", "content": full_prompt}
            ]
        )
        return (completion.choices[0].message)


    def answer_question(self, query):
        
        if self.db is None:
            print("Error: Database not initialized. Please call VectorDB method first.")
            return None
        
        new_docs = self.db.similarity_search(query)

        print("This is from answer_question fn:    ",new_docs)

        final_str=f'''Answer the query from the context given only
                    Context--{new_docs}
                    Query--{query}'''
        
        print(final_str)
        answer= self.__chat_with_memory(final_str)
        return answer["content"]

In [6]:
db_setup= DBSetup()

# db_setup.API_setter()

def creating_db(url):

    try:
        db_setup.YT_transcript(url)
        db_setup.VectorDB()
        status= "Success !"
        return status

    except Exception as e:
        status.update("Failed !")
        return f"Error in chat_bot1: {e}"


def answer_it(query):
    answer= db_setup.answer_question(query)
    audio= generate(answer, voice= 'pFZP5JQG7iQjIQuC4Bku')
    return answer, audio

In [8]:
with gr.Blocks() as demo:
    gr.Markdown('# YT Video Analyser')
    with gr.Tab("Upload the youtube link"):
            fn= creating_db
            text_input = gr.Textbox(placeholder="Give your YouTube link here", show_label=False)
            status = gr.Textbox(placeholder="Status", show_label=False)
            # text_output = gr.Textbox(placeholder="Status", show_label=False)
            text_buttons = gr.Button("Build the AI !!!")
            text_buttons.click(fn, inputs= text_input, outputs= status)
        

    with gr.Tab("chat with AI"):
        gr.Interface(
            fn=answer_it,
            inputs=["text"],
            outputs=[gr.Textbox(label="Answer"), gr.Audio(label="Answer in audio", autoplay= True)],
            live=False,
            title="Chat with me",
            description="Ask me anything regarding the video, I'm happy to talk with you",
            theme="compact",
            allow_flagging=False
        )

# Launch the Gradio interface
try:
    demo.queue().launch(debug=True, share=True)
except Exception as e:
    print(f"Error launching Gradio interface: {e}")

c:\Users\harik\anaconda3\envs\VectorDB_projects\lib\site-packages\gradio\blocks.py:525: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-65e30cb4-63cd70281e986b352628d4a5;50580f5a-e438-4699-bad9-706791aa8ef1)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
c:\Users\harik\anaconda3\envs\VectorDB_projects\lib\site-packages\gradio\interface.py:368: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://2fd0e484dcae6a131e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[Document(page_content='Back in 2015, Apple introduced the 12-inch\nMacBook. The first model ever to feature an Apple logo\nthat didn’t light up. And since then, the MacBook Pro and MacBook\nAir have also received the same treatment. This caused some controversy, since the glowing\nApple logo has been around for over two decades and has become an iconic design element of\nApple’s notebooks. So why was the illuminated Apple logo removed? Well, that’s exactly what I’ll explain\nright now. So when the 12-inch MacBook was introduced,\nit was the thinnest model in Apple’s history. In fact, at 13.1 millimeters, it was 24% thinner\nthan the MacBook Air. But that dramatic improvement didn’t come\nwithout compromises. For example, Apple had to create a new super-thin\nkeyboard that featured butterfly mechanisms instead of the traditional scissor design. They claimed the new keyboard was an improvement,\nbut that didn’t turn out to be the case. It was extremely faulty and resulted in a\nclass ac

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


This is from answer_question fn:     [Document(page_content='Back in 2015, Apple introduced the 12-inch\nMacBook. The first model ever to feature an Apple logo\nthat didn’t light up. And since then, the MacBook Pro and MacBook\nAir have also received the same treatment. This caused some controversy, since the glowing\nApple logo has been around for over two decades and has become an iconic design element of\nApple’s notebooks. So why was the illuminated Apple logo removed? Well, that’s exactly what I’ll explain\nright now. So when the 12-inch MacBook was introduced,\nit was the thinnest model in Apple’s history. In fact, at 13.1 millimeters, it was 24% thinner\nthan the MacBook Air. But that dramatic improvement didn’t come\nwithout compromises. For example, Apple had to create a new super-thin\nkeyboard that featured butterfly mechanisms instead of the traditional scissor design. They claimed the new keyboard was an improvement,\nbut that didn’t turn out to be the case. It was extreme

...

This is from answer_question fn:     [Document(page_content='Back in 2015, Apple introduced the 12-inch\nMacBook. The first model ever to feature an Apple logo\nthat didn’t light up. And since then, the MacBook Pro and MacBook\nAir have also received the same treatment. This caused some controversy, since the glowing\nApple logo has been around for over two decades and has become an iconic design element of\nApple’s notebooks. So why was the illuminated Apple logo removed? Well, that’s exactly what I’ll explain\nright now. So when the 12-inch MacBook was introduced,\nit was the thinnest model in Apple’s history. In fact, at 13.1 millimeters, it was 24% thinner\nthan the MacBook Air. But that dramatic improvement didn’t come\nwithout compromises. For example, Apple had to create a new super-thin\nkeyboard that featured butterfly mechanisms instead of the traditional scissor design. They claimed the new keyboard was an improvement,\nbut that didn’t turn out to be the case. It was extreme

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


This is from answer_question fn:     [Document(page_content='Back in 2015, Apple introduced the 12-inch\nMacBook. The first model ever to feature an Apple logo\nthat didn’t light up. And since then, the MacBook Pro and MacBook\nAir have also received the same treatment. This caused some controversy, since the glowing\nApple logo has been around for over two decades and has become an iconic design element of\nApple’s notebooks. So why was the illuminated Apple logo removed? Well, that’s exactly what I’ll explain\nright now. So when the 12-inch MacBook was introduced,\nit was the thinnest model in Apple’s history. In fact, at 13.1 millimeters, it was 24% thinner\nthan the MacBook Air. But that dramatic improvement didn’t come\nwithout compromises. For example, Apple had to create a new super-thin\nkeyboard that featured butterfly mechanisms instead of the traditional scissor design. They claimed the new keyboard was an improvement,\nbut that didn’t turn out to be the case. It was extreme

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


This is from answer_question fn:     [Document(page_content='Back in 2015, Apple introduced the 12-inch\nMacBook. The first model ever to feature an Apple logo\nthat didn’t light up. And since then, the MacBook Pro and MacBook\nAir have also received the same treatment. This caused some controversy, since the glowing\nApple logo has been around for over two decades and has become an iconic design element of\nApple’s notebooks. So why was the illuminated Apple logo removed? Well, that’s exactly what I’ll explain\nright now. So when the 12-inch MacBook was introduced,\nit was the thinnest model in Apple’s history. In fact, at 13.1 millimeters, it was 24% thinner\nthan the MacBook Air. But that dramatic improvement didn’t come\nwithout compromises. For example, Apple had to create a new super-thin\nkeyboard that featured butterfly mechanisms instead of the traditional scissor design. They claimed the new keyboard was an improvement,\nbut that didn’t turn out to be the case. It was extreme